## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

In [ ]:
application_df.NAME.value_counts()

In [ ]:
application_df1 = application_df.apply(lambda x: x.mask(x.map(x.value_counts())<100, 'other') if x.name=='NAME' else x)
application_df1.NAME.value_counts()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df1.drop(["EIN"], axis='columns')
application_df2.head()

In [ ]:
# Determine the number of unique values in each column.
application_df2.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df2.APPLICATION_TYPE.value_counts()

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T17', 'T15', 'T29','T14', 'T25', 'T2', 'T12', 'T13']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
clasval = application_df2.CLASSIFICATION.value_counts()
clasval

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
clasval2 = clasval[clasval >1]
clasval2

In [ ]:
clasval3 = clasval2[clasval2 <100]
clasval3

In [ ]:
application_df3 = application_df2.apply(lambda x: x.mask(x.map(x.value_counts())<100, 'other') if x.name=='CLASSIFICATION' else x)
application_df3.CLASSIFICATION.value_counts()

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# # Replace in dataframe
# for cls in classifications_to_replace:
#     application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# # Check to make sure binning was successful
# application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df4 = pd.get_dummies(application_df3)
application_df4.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
app_X = application_df4.drop(columns='IS_SUCCESSFUL')
app_y = application_df4['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(app_X, app_y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu", input_dim=81))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=12, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save('model.h5')